In [6]:
### Paso 1: Cargar estructuras WT y Mutante desde carpetas ###
# Estructuras en subdirectorios tipo /path/to/WT/*/*.pdb y /path/to/MUT/*/*.pdb

### Paso 2: Superposición sobre residuos CU ###
# Usar MDTraj para alinear todas las estructuras sobre los mismos residuos

### Paso 3: Generar un único PDB con:
# - Una sola proteína (de referencia, por ejemplo la del WT)
# - Todos los DMPs (uno por estructura) coloreados por tipo: azul (WT) o verde (Mutante)

### Paso 4: Filtrar poses redundantes (RMSD < 3 Å) entre WT y MUT ###
# - Comparar todos los DMPs mutantes vs. los del WT
# - Si RMSD < 3.0 → se consideran superpuestos y se excluyen de la segunda salida

### Paso 5: Guardar el nuevo PDB con:
# - Proteína de referencia
# - Solo DMPs que no se superpongan (RMSD > 3 Å)
# - Colores WT (azul) / Mutante (verde)


In [7]:
import mdtraj as md
import glob
import os

# Paths
wt_path_pattern = "/HDD/sromero/MCOA/WT/PH8/DOCK_sonia/global/protonat/analisis/structures_BE_-5.5_DistDMPT1_0.0to20.0/*/*.pdb"
mut_path_pattern = "/HDD/sromero/MCOA/15E6loop5/DOCK_DMP/PH8/global/DMP_protonated/analysis/structures_BE_-5.5_DistDMPT1_0.0to20.0/*/*.pdb"

# Obtener lista de archivos
wt_files = sorted(glob.glob(wt_path_pattern))
mut_files = sorted(glob.glob(mut_path_pattern))

# Cargar estructuras en memoria
wt_structures = []
mut_structures = []

print(f"📁 Cargando {len(wt_files)} estructuras del WT...")
for f in wt_files:
    traj = md.load(f)
    wt_structures.append((os.path.basename(f), traj))

print(f"📁 Cargando {len(mut_files)} estructuras del MUTANTE...")
for f in mut_files:
    traj = md.load(f)
    mut_structures.append((os.path.basename(f), traj))

print(f"✅ Total estructuras cargadas: {len(wt_structures)} WT | {len(mut_structures)} Mutante")


📁 Cargando 46 estructuras del WT...
📁 Cargando 165 estructuras del MUTANTE...
✅ Total estructuras cargadas: 46 WT | 165 Mutante


In [8]:
def get_cu_indices(traj):
    return traj.topology.select("resname CU")
ref_name, ref_traj = wt_structures[0]
cu_indices_ref = get_cu_indices(ref_traj)
if len(cu_indices_ref) == 0:
    raise ValueError("❌ No se encontraron átomos de CU en la referencia.")
print(f"🧬 Usando '{ref_name}' como estructura de referencia (WT)")
print(f"🔍 Nº de átomos CU en referencia: {len(cu_indices_ref)}")
# Superponer WT
wt_aligned = []
for name, traj in wt_structures:
    indices = get_cu_indices(traj)
    if len(indices) != len(cu_indices_ref):
        print(f"⚠️ {name}: Nº de CU diferente ({len(indices)} vs {len(cu_indices_ref)}). Saltando.")
        continue
    aligned = traj.superpose(ref_traj, atom_indices=indices, ref_atom_indices=cu_indices_ref)
    wt_aligned.append((name, aligned))
# Superponer MUT
mut_aligned = []
for name, traj in mut_structures:
    indices = get_cu_indices(traj)
    if len(indices) != len(cu_indices_ref):
        print(f"⚠️ {name}: Nº de CU diferente ({len(indices)} vs {len(cu_indices_ref)}). Saltando.")
        continue
    aligned = traj.superpose(ref_traj, atom_indices=indices, ref_atom_indices=cu_indices_ref)
    mut_aligned.append((name, aligned))
print(f"✅ Superposición completada: {len(wt_aligned)} WT | {len(mut_aligned)} Mutante")

🧬 Usando 'wt_ph8_sonia_rep1-0004_001.pdb' como estructura de referencia (WT)
🔍 Nº de átomos CU en referencia: 4
✅ Superposición completada: 46 WT | 165 Mutante


In [9]:
import mdtraj as md
import numpy as np
from itertools import combinations
import os

def extract_ligand_coords(traj, ligand_name="DMP"):
    """Extrae las coordenadas del ligando especificado"""
    ligand_indices = traj.topology.select(f"resname {ligand_name}")
    if len(ligand_indices) == 0:
        return None
    return traj.xyz[0, ligand_indices]  # Primera frame

def create_combined_pdb_with_chains(ref_traj, wt_aligned, mut_aligned, 
                                   output_file="combined_structures_BE-5.5.pdb",
                                   ligand_name="DMP"):
    """
    Crea un PDB combinado con todos los ligandos DMP
    - Cadena W para ligandos del WT
    - Cadena M para ligandos del Mutante
    """
    
    # Empezar con la estructura de referencia (solo proteína)
    protein_indices = ref_traj.topology.select("protein or resname CU")
    protein_traj = ref_traj.atom_slice(protein_indices)
    
    # Filtrar estructuras que tienen ligandos
    wt_with_ligands = []
    mut_with_ligands = []
    
    for name, traj in wt_aligned:
        ligand_indices = traj.topology.select(f"resname {ligand_name}")
        if len(ligand_indices) > 0:
            wt_with_ligands.append((name, traj))
    
    for name, traj in mut_aligned:
        ligand_indices = traj.topology.select(f"resname {ligand_name}")
        if len(ligand_indices) > 0:
            mut_with_ligands.append((name, traj))
    
    # Escribir el archivo PDB
    write_combined_pdb(output_file, protein_traj, wt_with_ligands, mut_with_ligands)
    
    print(f"✅ PDB combinado creado: {output_file}")
    print(f"   - Ligandos WT (cadena W): {len(wt_with_ligands)}")
    print(f"   - Ligandos Mutante (cadena M): {len(mut_with_ligands)}")
    
    return len(wt_with_ligands), len(mut_with_ligands)

def write_combined_pdb(filename, protein_traj, wt_ligands_data, mut_ligands_data):
    """Escribe el archivo PDB combinado"""
    with open(filename, 'w') as f:
        f.write("REMARK Combined protein structure with WT and Mutant ligands\n")
        f.write(f"REMARK WT ligands: {len(wt_ligands_data)} (chain W)\n")
        f.write(f"REMARK Mutant ligands: {len(mut_ligands_data)} (chain M)\n")
        
        atom_num = 1


        # Escribir átomos de la proteína
        for i, atom in enumerate(protein_traj.topology.atoms):
            coord = protein_traj.xyz[0, i]
            chain_id = atom.residue.chain.chain_id or "A"
            element_symbol = (atom.element.symbol if atom.element is not None else " X").rjust(2)
    
            f.write(f"ATOM  {atom_num:5d} {atom.name:<4s} {atom.residue.name:>3s} {chain_id:1s}"
                    f"{atom.residue.resSeq:4d}    "
                    f"{coord[0]*10:8.3f}{coord[1]*10:8.3f}{coord[2]*10:8.3f}"
                    f"{1.00:6.2f}{20.00:6.2f}          {element_symbol}\n")
            atom_num += 1

        # Escribir ligandos WT (cadena W)
        for res_num, (name, traj) in enumerate(wt_ligands_data, 1):
            ligand_indices = traj.topology.select("resname DMP")
            if len(ligand_indices) > 0:
                for idx in ligand_indices:
                    atom = traj.topology.atom(idx)
                    coord = traj.xyz[0, idx]
                    element_symbol = (atom.element.symbol if atom.element is not None else " X").rjust(2)

                    f.write(f"ATOM  {atom_num:5d} {atom.name:<4s} DMP W{res_num:4d}    "
                            f"{coord[0]*10:8.3f}{coord[1]*10:8.3f}{coord[2]*10:8.3f}"
                            f"{1.00:6.2f}{20.00:6.2f}          {element_symbol}\n")
                    atom_num += 1
    
                # Escribir TER después del residuo DMP
                f.write(f"TER   {atom_num:5d}      DMP W{res_num:4d}\n")
                atom_num += 1

        # Escribir ligandos del mutante (cadena M)
        for res_num, (name, traj) in enumerate(mut_ligands_data, 1):
            ligand_indices = traj.topology.select("resname DMP")
            if len(ligand_indices) > 0:
                for idx in ligand_indices:
                    atom = traj.topology.atom(idx)
                    coord = traj.xyz[0, idx]
                    element_symbol = (atom.element.symbol if atom.element is not None else " X").rjust(2)
    
                    f.write(f"ATOM  {atom_num:5d} {atom.name:<4s} DMP M{res_num:4d}    "
                            f"{coord[0]*10:8.3f}{coord[1]*10:8.3f}{coord[2]*10:8.3f}"
                            f"{1.00:6.2f}{20.00:6.2f}          {element_symbol}\n")
                    atom_num += 1

                # Escribir TER después del residuo DMP
                f.write(f"TER   {atom_num:5d}      DMP M{res_num:4d}\n")
                atom_num += 1

        # Finalizar archivo
        f.write("END\n")

def calculate_ligand_rmsd(coords1, coords2):
    """Calcula RMSD entre dos conjuntos de coordenadas de ligandos"""
    if coords1 is None or coords2 is None:
        return float('inf')
    
    # Asegurar que ambos conjuntos tengan el mismo número de átomos
    if len(coords1) != len(coords2):
        return float('inf')
    
    # Calcular RMSD
    diff = coords1 - coords2
    rmsd = np.sqrt(np.mean(np.sum(diff**2, axis=1))) *10
    return rmsd

def filter_non_overlapping_ligands(wt_aligned, mut_aligned, rmsd_threshold=2.0, 
                                  ligand_name="DMP"):
    """
    Filtra ligandos que no se superpongan entre WT y Mutante
    Mantiene tanto ligandos WT como Mutante que no se solapen entre sí
    
    Args:
        wt_aligned: Lista de estructuras WT alineadas
        mut_aligned: Lista de estructuras Mutante alineadas
        rmsd_threshold: Umbral de RMSD para considerar solapamiento (Å)
        ligand_name: Nombre del ligando a analizar
    
    Returns:
        wt_filtered, mut_filtered: Listas filtradas de estructuras
    """
    
    # Extraer coordenadas de todos los ligandos
    wt_ligands = []
    mut_ligands = []
    
    for name, traj in wt_aligned:
        coords = extract_ligand_coords(traj, ligand_name)
        if coords is not None:
            wt_ligands.append((name, traj, coords))
    
    for name, traj in mut_aligned:
        coords = extract_ligand_coords(traj, ligand_name)
        if coords is not None:
            mut_ligands.append((name, traj, coords))
    
    print(f"🔍 Analizando solapamiento entre {len(wt_ligands)} ligandos WT y {len(mut_ligands)} ligandos Mutante")
    
    # Crear matriz de RMSD entre todos los pares WT-Mutante
    overlapping_pairs = []
    
    for i, (wt_name, wt_traj, wt_coords) in enumerate(wt_ligands):
        for j, (mut_name, mut_traj, mut_coords) in enumerate(mut_ligands):
            rmsd = calculate_ligand_rmsd(wt_coords, mut_coords)
            
            if rmsd < rmsd_threshold:
                overlapping_pairs.append((i, j, rmsd, wt_name, mut_name))
                print(f"⚠️ Solapamiento detectado: {wt_name} vs {mut_name} (RMSD: {rmsd:.2f} Å)")
    
    # Identificar índices de ligandos que se solapan
    overlapping_wt_indices = set()
    overlapping_mut_indices = set()
    
    for wt_idx, mut_idx, rmsd, wt_name, mut_name in overlapping_pairs:
        overlapping_wt_indices.add(wt_idx)
        overlapping_mut_indices.add(mut_idx)
    
    # Filtrar ligandos que NO se solapan
    wt_filtered = []
    mut_filtered = []
    
    # Mantener ligandos WT que no se solapan con ningún mutante
    for i, (wt_name, wt_traj, wt_coords) in enumerate(wt_ligands):
        if i not in overlapping_wt_indices:
            wt_filtered.append((wt_name, wt_traj))
    
    # Mantener ligandos Mutante que no se solapan con ningún WT
    for j, (mut_name, mut_traj, mut_coords) in enumerate(mut_ligands):
        if j not in overlapping_mut_indices:
            mut_filtered.append((mut_name, mut_traj))
    
    print(f"✅ Filtrado completado:")
    print(f"   - Ligandos WT originales: {len(wt_ligands)}")
    print(f"   - Ligandos WT no solapantes: {len(wt_filtered)}")
    print(f"   - Ligandos WT eliminados por solapamiento: {len(wt_ligands) - len(wt_filtered)}")
    print(f"   - Ligandos Mutante originales: {len(mut_ligands)}")
    print(f"   - Ligandos Mutante no solapantes: {len(mut_filtered)}")
    print(f"   - Ligandos Mutante eliminados por solapamiento: {len(mut_ligands) - len(mut_filtered)}")
    print(f"   - Total de pares solapantes eliminados: {len(overlapping_pairs)}")
    
    return wt_filtered, mut_filtered

def create_non_overlapping_pdb(ref_traj, wt_filtered, mut_filtered, 
                              output_file="non_overlapping_structures_BE-5.5.pdb",
                              ligand_name="DMP"):
    """
    Crea un PDB con ligandos no solapantes
    """
    # Empezar con la estructura de referencia (solo proteína)
    protein_indices = ref_traj.topology.select("protein or resname CU")
    protein_traj = ref_traj.atom_slice(protein_indices)
    
    # Escribir el archivo PDB
    write_combined_pdb(output_file, protein_traj, wt_filtered, mut_filtered)
    
    print(f"✅ PDB filtrado creado: {output_file}")
    print(f"   - Ligandos WT no solapantes: {len(wt_filtered)}")
    print(f"   - Ligandos Mutante no solapantes: {len(mut_filtered)}")
    
    return len(wt_filtered), len(mut_filtered)

# Ejemplo de uso:
if __name__ == "__main__":
    # Asumiendo que ya tienes ref_traj, wt_aligned, mut_aligned del código anterior
    
    # Paso 3: Crear PDB combinado con todos los ligandos
    print("🔄 Paso 3: Creando PDB combinado con todos los ligandos...")
    wt_count, mut_count = create_combined_pdb_with_chains(
        ref_traj, wt_aligned, mut_aligned, 
        "all_ligands_combined_BE-5.5.pdb"
    )
    
    # Paso 4: Filtrar ligandos no solapantes
    print("\n🔄 Paso 4: Filtrando ligandos no solapantes...")
    wt_filtered, mut_filtered = filter_non_overlapping_ligands(
        wt_aligned, mut_aligned, 
        rmsd_threshold=2.0  # Ajusta según necesites
    )
    
    # Crear PDB con ligandos filtrados
    create_non_overlapping_pdb(
        ref_traj, wt_filtered, mut_filtered,
        "non_overlapping_ligands_BE-5.5.pdb"
    )

🔄 Paso 3: Creando PDB combinado con todos los ligandos...
✅ PDB combinado creado: all_ligands_combined_BE-5.5.pdb
   - Ligandos WT (cadena W): 46
   - Ligandos Mutante (cadena M): 165

🔄 Paso 4: Filtrando ligandos no solapantes...
🔍 Analizando solapamiento entre 46 ligandos WT y 165 ligandos Mutante
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0084_001.pdb vs 15e6p804-048_001.pdb (RMSD: 1.21 Å)
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0084_001.pdb vs 15e6p804-055_001.pdb (RMSD: 1.33 Å)
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0084_001.pdb vs 15e6p804-091_001.pdb (RMSD: 1.50 Å)
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0084_001.pdb vs 15e6p804-094_001.pdb (RMSD: 1.84 Å)
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0084_001.pdb vs 15e6p804-098_001.pdb (RMSD: 1.43 Å)
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0086_001.pdb vs 15e6p803-002_001.pdb (RMSD: 1.59 Å)
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0086_001.pdb vs 15e6p804-050_001.pdb (RMSD: 1.39 Å)
⚠️ Solapamiento dete

In [13]:
    # Paso 4: Filtrar ligandos no solapantes
    print("\n🔄 Paso 4: Filtrando ligandos no solapantes...")
    wt_filtered, mut_filtered = filter_non_overlapping_ligands(
        wt_aligned, mut_aligned, 
        rmsd_threshold=3.0  # Ajusta según necesites
    )
    
    # Crear PDB con ligandos filtrados
    create_non_overlapping_pdb(
        ref_traj, wt_filtered, mut_filtered,
        "non_overlapping_ligands_BE-5.5_rmsd_3.0.pdb"
    )


🔄 Paso 4: Filtrando ligandos no solapantes...
🔍 Analizando solapamiento entre 46 ligandos WT y 165 ligandos Mutante
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0020_002.pdb vs 15e6p801-039_001.pdb (RMSD: 2.16 Å)
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0020_002.pdb vs 15e6p804-010_001.pdb (RMSD: 2.54 Å)
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0020_002.pdb vs 15e6p804-014_001.pdb (RMSD: 2.21 Å)
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0020_002.pdb vs 15e6p804-017_001.pdb (RMSD: 2.32 Å)
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0024_001.pdb vs 15e6p802-009_001.pdb (RMSD: 2.76 Å)
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0027_001.pdb vs 15e6p802-009_001.pdb (RMSD: 2.79 Å)
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0084_001.pdb vs 15e6p802-023_001.pdb (RMSD: 2.43 Å)
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0084_001.pdb vs 15e6p802-096_002.pdb (RMSD: 2.51 Å)
⚠️ Solapamiento detectado: wt_ph8_sonia_rep1-0084_001.pdb vs 15e6p803-052_002.pdb (RMSD: 2.93 Å)
⚠️ Solapam

(28, 97)